In [35]:
import pandas as pd
import altair as alt

alt.data_transformers.disable_max_rows()

DATA_URL = "https://github.com/UIUC-iSchool-DataViz/is445_data/raw/main/licenses_fall2022.csv"
df = pd.read_csv(DATA_URL)
df.head()

,_id,License Type,Description,License Number,License Status,Business,Title,First Name,Middle,Last Name,...,Specialty/Qualifier,Controlled Substance Schedule,Delegated Controlled Substance Schedule,Ever Disciplined,LastModifiedDate,Case Number,Action,Discipline Start Date,Discipline End Date,Discipline Reason
0,1189509,DETECTIVE BOARD,PERMANENT EMPLOYEE REGISTRATION,129446286,NOT RENEWED,N,NaN,EILEEN,NaN,SANTACRUZ,...,NaN,NaN,NaN,N,03/18/2022,NaN,NaN,NaN,NaN,NaN
1,801037,DETECTIVE BOARD,FIREARM CONTROL CARD,229030294.0,NOT RENEWED,N,NaN,DAGMAR,J,NORDLUND,...,NaN,NaN,NaN,N,08/16/2006,NaN,NaN,NaN,NaN,NaN
2,365129,COSMO,LICENSED COSMETOLOGIST,11053076.0,NOT RENEWED,N,NaN,RADOJE,NaN,ZELENOVIC,...,NaN,NaN,NaN,N,05/26/2006,NaN,NaN,NaN,NaN,NaN
3,595427,COSMO,LICENSED COSMETOLOGIST,11295645.0,ACTIVE,N,NaN,BECKY SUE,L,BURROUGHS,...,NaN,NaN,NaN,N,11/12/2021,NaN,NaN,NaN,NaN,NaN
4,653668,COSMO,LICENSED NAIL TECHNICIAN,169006247,NOT RENEWED,N,NaN,BILL G,L,LETNER,...,NaN,NaN,NaN,N,05/30/2006,NaN,NaN,NaN,NaN,NaN


In [36]:
df.shape, df.columns.tolist()[:20]

((10000, 31),
 ['_id',
  'License Type',
  'Description',
  'License Number',
  'License Status',
  'Business',
  'Title',
  'First Name',
  'Middle',
  'Last Name',
  'Prefix',
  'Suffix',
  'Business Name',
  'BusinessDBA',
  'Original Issue Date',
  'Effective Date',
  'Expiration Date',
  'City',
  'State',
  'Zip'])

In [37]:
df = df.copy()
df.columns = (
    df.columns
      .str.strip()
      .str.lower()
      .str.replace(r'\s+', '_', regex=True)
      .str.replace(r'[^0-9a-zA-Z_]', '', regex=True)
)
df.columns.tolist()[:30]


['_id',
 'license_type',
 'description',
 'license_number',
 'license_status',
 'business',
 'title',
 'first_name',
 'middle',
 'last_name',
 'prefix',
 'suffix',
 'business_name',
 'businessdba',
 'original_issue_date',
 'effective_date',
 'expiration_date',
 'city',
 'state',
 'zip',
 'county',
 'specialtyqualifier',
 'controlled_substance_schedule',
 'delegated_controlled_substance_schedule',
 'ever_disciplined',
 'lastmodifieddate',
 'case_number',
 'action',
 'discipline_start_date',
 'discipline_end_date']

In [38]:
def show_top(col, n=15):
    if col in df.columns:
        print(f"\n== {col} ==")
        display(df[col].value_counts(dropna=False).head(n))
    else:
        print(f"{col} not found")

candidates = [
    'license_type','licensetype','type',
    'license_status','status',
    'state','st','jurisdiction'
]
for c in candidates:
    show_top(c)



== license_type ==


,count
license_type,
DETECTIVE BOARD,4867
COSMO,3781
DENTAL,739
FUNERAL AND EMBALMER,98
DIETETIC AND NUTRITION,73
DESIGN FIRM,71
MASSAGE LICENSING BD,52
HOME INSPECTOR,46
COMM ASSOC MGR,37


licensetype not found
type not found

== license_status ==


,count
license_status,
NOT RENEWED,6331
ACTIVE,2440
INACTIVE,532
TERMINATED CARD RETURNED,431
CANCELLED,78
DECEASED,60
TERMINATED WITHOUT CARD,32
EXPIRED,21
CLOSED,21


status not found

== state ==


,count
state,
IL,9026
IN,139
WI,94
MO,91
FL,86
CA,84
TX,62
IA,55
MI,30


st not found
jurisdiction not found


In [39]:
date_like = [c for c in df.columns if any(k in c for k in ['date','issued','effective','start','expire','end'])]
date_like

['original_issue_date',
 'effective_date',
 'expiration_date',
 'lastmodifieddate',
 'discipline_start_date',
 'discipline_end_date']

In [40]:
import pandas as pd

for c in date_like:
    try:
        df[c] = pd.to_datetime(df[c], errors='coerce')
    except Exception as e:
        print(f"Could not parse {c}: {e}")

# Pick a primary date for time series
priority = ['effective_date','issued_date','issue_date','start_date','date']
primary_date = next((c for c in priority if c in df.columns), None)
primary_date


'effective_date'

In [41]:
import pandas as pd
from IPython.display import display

# 1) Re-detect date-like columns
date_like = [c for c in df.columns if any(k in c for k in ['date','issued','issue','effective','start','expire','end'])]
print("Date-like columns:", date_like)

# 2) Try parsing each to datetime; track how many valid (non-NaN) dates we get
parsed_nonnull = {}
for c in date_like:
    s = pd.to_datetime(df[c], errors='coerce', infer_datetime_format=True)
    df[c + '_parsed'] = s
    parsed_nonnull[c] = int(s.notna().sum())

print("Non-null parsed counts per column:", parsed_nonnull)

# 3) Pick the column with the most valid dates as primary
primary_date = None
if parsed_nonnull:
    # choose the date column with the max non-null rows
    candidate = max(parsed_nonnull, key=parsed_nonnull.get)
    if parsed_nonnull[candidate] > 0:
        primary_date = candidate + '_parsed'

print("Chosen primary_date column:", primary_date)

# 4) Create year_month and show a preview
if primary_date:
    df['year_month'] = df[primary_date].dt.to_period('M').astype(str)
    display(df[[primary_date, 'year_month']].head(10))
else:
    print("Still no usable date column. If you see a 'year'/'month' column, we can build year_month from those.")


Date-like columns: ['original_issue_date', 'effective_date', 'expiration_date', 'lastmodifieddate', 'discipline_start_date', 'discipline_end_date']
Non-null parsed counts per column: {'original_issue_date': 9995, 'effective_date': 9208, 'expiration_date': 9330, 'lastmodifieddate': 10000, 'discipline_start_date': 343, 'discipline_end_date': 138}
Chosen primary_date column: lastmodifieddate_parsed


/tmp/ipython-input-2161010468.py:11: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  s = pd.to_datetime(df[c], errors='coerce', infer_datetime_format=True)
/tmp/ipython-input-2161010468.py:11: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  s = pd.to_datetime(df[c], errors='coerce', infer_datetime_format=True)
/tmp/ipython-input-2161010468.py:11: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsi

,lastmodifieddate_parsed,year_month
0,2022-03-18,2022-03
1,2006-08-16,2006-08
2,2006-05-26,2006-05
3,2021-11-12,2021-11
4,2006-05-30,2006-05
5,2022-05-09,2022-05
6,2006-05-26,2006-05
7,2007-01-19,2007-01
8,2018-07-19,2018-07
9,2006-05-23,2006-05


In [42]:
# pick the closest-matching column names
type_col = [c for c in df.columns if 'type' in c][0]
status_col = [c for c in df.columns if 'status' in c][0]

summary = (
    df.groupby([type_col, status_col])
      .size()
      .reset_index(name='count')
)

chart1 = alt.Chart(summary).mark_bar().encode(
    x=alt.X(f'{type_col}:N', sort='-y', title='License Type'),
    y=alt.Y('count:Q', title='Number of Licenses'),
    color=alt.Color(f'{status_col}:N', title='License Status'),
    tooltip=[type_col, status_col, 'count']
).properties(
    title='Counts of Licenses by Type and Status',
    width=700,
    height=400
)

chart1


alt.Chart(...)

In [43]:
# Reuse column names
type_col = [c for c in df.columns if 'type' in c][0]

# Aggregate monthly counts per license type
time_summary = (
    df.groupby(['year_month', type_col])
      .size()
      .reset_index(name='count')
      .dropna(subset=['year_month'])
)

# Create a selection parameter for dropdown
type_dropdown = alt.param(
    name='SelectedType',
    bind=alt.binding_select(
        options=sorted(time_summary[type_col].unique()),
        name='Select License Type: '
    ),
    value="COMM ASSOC MGR"
)

# Define the chart
chart2 = (
    alt.Chart(time_summary)
    .mark_line(point=True)
    .encode(
        x=alt.X('year_month:N', title='Year-Month'),
        y=alt.Y('count:Q', title='Number of Licenses'),
        color=alt.Color(f'{type_col}:N', legend=None),
        tooltip=['year_month', type_col, 'count']
    )
    .add_params(type_dropdown)
    .transform_filter(f"datum.{type_col} == SelectedType")
    .properties(
        title='Number of Licenses Over Time by Type (Interactive)',
        width=700,
        height=400
    )
)

chart2

alt.Chart(...)